## bert score
- https://github.com/Tiiiger/bert_score

In [ ]:
!pip install bert-score

In [ ]:
from bert_score import score

In [ ]:
reference_text = ["hello iam your assistant, any help ?",
                  "the answer based on your question is 10"]

generated_text = ["yes", "no"]

In [ ]:
P, R, F1 = score(generated_text,
                 reference_text,
                 lang='en',
                 verbose=True,
                 model_type='microsoft/deberta-xlarge-mnli')

In [ ]:
P, R, F1

(tensor([0.5946, 0.3906]), tensor([0.3686, 0.3669]), tensor([0.4551, 0.3783]))

In [ ]:
reference_text = ["hello iam your assistant, any help ?",
                  "the answer based on your question is 10"]

generated_text = ["yes my name is adipta martulandi", "no, the answer is 15"]

In [ ]:
P, R, F1 = score(generated_text,
                 reference_text,
                 lang='en',
                 verbose=True,
                 model_type='microsoft/deberta-xlarge-mnli')

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.15 seconds, 13.41 sentences/sec


In [ ]:
P, R, F1

(tensor([0.5040, 0.7216]), tensor([0.5529, 0.6311]), tensor([0.5273, 0.6733]))

In [ ]:
reference_text = ["halo ada yang bisa aku bantu ? namaku adipta",
                  "jawabanya itu 10"]

generated_text = ["aku adalah asistant", "5+5+5 hasilnya 15"]

In [ ]:
P, R, F1 = score(generated_text,
                 reference_text,
                 lang='id',
                 verbose=True,
                 model_type='microsoft/deberta-xlarge-mnli')

In [ ]:
P, R, F1

(tensor([0.6633, 0.4962]), tensor([0.6357, 0.6379]), tensor([0.6492, 0.5582]))

In [ ]:
reference_text = ["halo ada yang bisa aku bantu ? namaku adipta",
                  "jawabanya itu 15"]

generated_text = ["aku adalah asistant yang bernama adipta", "5+5+5 hasilnya 15"]

In [ ]:
P, R, F1 = score(generated_text,
                 reference_text,
                 lang='id',
                 verbose=True,
                 model_type='microsoft/deberta-xlarge-mnli')

In [ ]:
P, R, F1

(tensor([0.7229, 0.5073]), tensor([0.6736, 0.6503]), tensor([0.6974, 0.5700]))

## bleurt
- https://github.com/google-research/bleurt
- https://huggingface.co/spaces/evaluate-metric/bleurt

In [ ]:
# !pip install --upgrade pip  # ensures that pip is current
# !git clone https://github.com/google-research/bleurt.git
# !cd bleurt

In [ ]:
# !pip install bleurt/bleurt
# !pip install evaluate

seems harus install di local

## self check gpt
- https://www.confident-ai.com/blog/llm-evaluation-metrics-everything-you-need-for-llm-evaluation
- https://github.com/potsawee/selfcheckgpt?tab=readme-ov-file

In [ ]:
!pip install selfcheckgpt

In [ ]:
# Include necessary packages (torch, spacy, ...)
from selfcheckgpt.modeling_selfcheck import SelfCheckMQAG, SelfCheckBERTScore, SelfCheckNgram
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
selfcheck_mqag = SelfCheckMQAG(device=device) # set device to 'cuda' if GPU is available
selfcheck_bertscore = SelfCheckBERTScore(rescale_with_baseline=True)
selfcheck_ngram = SelfCheckNgram(n=1) # n=1 means Unigram, n=2 means Bigram, etc.

In [ ]:
# pip install -U spacy
# python -m spacy download en_core_web_sm
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

In [ ]:
# LLM's text (e.g. GPT-3 response) to be evaluated at the sentence level  & Split it into sentences
passage = "Michael Alan Weiner (born March 31, 1942) is an American radio host. He is the host of The Savage Nation."
sentences = [sent.text.strip() for sent in nlp(passage).sents] # spacy sentence tokenization
print(sentences)
['Michael Alan Weiner (born March 31, 1942) is an American radio host.', 'He is the host of The Savage Nation.']

# Other samples generated by the same LLM to perform self-check for consistency
sample1 = "Michael Alan Weiner (born March 31, 1942) is an American radio host. He is the host of The Savage Country."
sample2 = "Michael Alan Weiner (born January 13, 1960) is a Canadian radio host. He works at The New York Times."
sample3 = "Michael Alan Weiner (born March 31, 1942) is an American radio host. He obtained his PhD from MIT."

['Michael Alan Weiner (born March 31, 1942) is an American radio host.', 'He is the host of The Savage Nation.']


In [ ]:
# --------------------------------------------------------------------------------------------------------------- #
# SelfCheck-MQAG: Score for each sentence where value is in [0.0, 1.0] and high value means non-factual
# Additional params for each scoring_method:
# -> counting: AT (answerability threshold, i.e. questions with answerability_score < AT are rejected)
# -> bayes: AT, beta1, beta2
# -> bayes_with_alpha: beta1, beta2
sent_scores_mqag = selfcheck_mqag.predict(
    sentences = sentences,               # list of sentences
    passage = passage,                   # passage (before sentence-split)
    sampled_passages = [sample1, sample2, sample3], # list of sampled passages
    num_questions_per_sent = 5,          # number of questions to be drawn
    scoring_method = 'bayes_with_alpha', # options = 'counting', 'bayes', 'bayes_with_alpha'
    beta1 = 0.8, beta2 = 0.8,            # additional params depending on scoring_method
)
print(sent_scores_mqag)

[0.28782231 0.26767984]


In [ ]:
# --------------------------------------------------------------------------------------------------------------- #
# SelfCheck-BERTScore: Score for each sentence where value is in [0.0, 1.0] and high value means non-factual
sent_scores_bertscore = selfcheck_bertscore.predict(
    sentences = sentences,                          # list of sentences
    sampled_passages = [sample1, sample2, sample3], # list of sampled passages
)
print(sent_scores_bertscore)

In [ ]:
# --------------------------------------------------------------------------------------------------------------- #
# SelfCheck-Ngram: Score at sentence- and document-level where value is in [0.0, +inf) and high value means non-factual
# as opposed to SelfCheck-MQAG and SelfCheck-BERTScore, SelfCheck-Ngram's score is not bounded
sent_scores_ngram = selfcheck_ngram.predict(
    sentences = sentences,
    passage = passage,
    sampled_passages = [sample1, sample2, sample3],
)
print(sent_scores_ngram)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


{'sent_level': {'avg_neg_logprob': [3.184312427726157, 3.279774864365169], 'max_neg_logprob': [3.4760986898352733, 4.574710978503383]}, 'doc_level': {'avg_neg_logprob': 3.218678904916201, 'avg_max_neg_logprob': 4.025404834169328}}


In [ ]:
from selfcheckgpt.modeling_selfcheck import SelfCheckNLI
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_nli = SelfCheckNLI(device=device) # set device to 'cuda' if GPU is available

sent_scores_nli = selfcheck_nli.predict(
    sentences = sentences,                          # list of sentences
    sampled_passages = [sample1, sample2, sample3], # list of sampled passages
)
print(sent_scores_nli)

## G-Eval
- https://github.com/confident-ai/deepeval

In [ ]:
!pip install deepeval

In [2]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [3]:
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval, HallucinationMetric, AnswerRelevancyMetric
from deepeval import assert_test

In [7]:
result_gpt = "Michael Alan Weiner (born March 31, 1942) is an American radio host."
actual = "Michael Alan Weiner (born March 31, 1942) is an American radio host."

In [8]:
test_case = LLMTestCase(input=result_gpt, actual_output=actual)

In [9]:
coherence_metric = GEval(
    name="Correctness",
    criteria="Determine whether the input sentence is correct based on the expected/actual output.",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

coherence_metric.measure(test_case)
print(coherence_metric.score)
print(coherence_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.872966847594936
The actual output is grammatically correct and conveys the intended meaning. It has proper punctuation and capitalization but lacks additional context or detail that might be expected.


In [10]:
coherence_metric = GEval(
    name="Similarity",
    criteria="Determine whether the input sentence is similar on the expected/actual output based on the semantic meaning",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

coherence_metric.measure(test_case)
print(coherence_metric.score)
print(coherence_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.241009902716047
Mentions Michael Alan Weiner and his birthdate but lacks further detail and context about his career.


In [ ]:


hallucination_metric = HallucinationMetric()
test_case = LLMTestCase(
  input="What if these shoes don't fit?",
  actual_output="We offer a 30-day full refund at no extra costs.",
  context=["All customers are eligible for a 30 day full refund at no extra costs."]
  )

hallucination_metric.measure(test_case)

Output()

0.0

In [19]:
answer_relevancy_metric = AnswerRelevancyMetric()
test_case = LLMTestCase(
  input="my name is adipta martulandi",
  actual_output="my name is agung"
)

In [20]:
answer_relevancy_metric.measure(test_case)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [21]:
answer_relevancy_metric.score

0.0

In [ ]:
answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.7)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    # Replace this with the actual output from your LLM application
    actual_output="We offer a 30-day full refund at no extra costs.",
    retrieval_context=["All customers are eligible for a 30 day full refund at no extra costs."]
)

answer_relevancy_metric.measure(test_case)
print(answer_relevancy_metric.score)
# Most metrics also offer an explanation
print(answer_relevancy_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0
The score is 1.00 because the response perfectly addresses the concern about the shoes potentially not fitting, without any irrelevant information. Great job!


## falcon evaluate
- https://github.com/Praveengovianalytics/falcon-evaluate

In [22]:
!pip install falcon_evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 84.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 wh

In [ ]:
from falcon_evaluate.fevaluate_results import ModelScoreSummary
from falcon_evaluate.fevaluate_plot import ModelPerformancePlotter
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Make sure that your validation dataframe should have "prompt" & "reference" column
# & rest other columns are model generated responses
df = pd.DataFrame({
    'prompt': [
        "What is the capital of France?"
    ],
    'reference': [
        "The capital of France is Paris."
    ],
    'Model A': [
        "Paris is the capital of France."
    ],
    'Model B': [
        "Capital of France is Paris."
    ],
    'Model C': [
        "Capital of France was Paris."
    ],
})

In [ ]:
df

,prompt,reference,Model A,Model B,Model C
0,What is the capital of France?,The capital of France is Paris.,Paris is the capital of France.,Capital of France is Paris.,Capital of France was Paris.


In [ ]:
model_score_summary = ModelScoreSummary(df)
result, agg_score_df = model_score_summary.execute_summary()

In [ ]:
result

,prompt,reference,Model A,Model A-Scores,Model A-falcon_Score,Model B,Model B-Scores,Model B-falcon_Score,Model C,Model C-Scores,Model C-falcon_Score
0,What is the capital of France?,The capital of France is Paris.,Paris is the capital of France.,"{'Readability and Complexity': {'ARI': 2.0, 'F...",{'Readability and Complexity': {'Arithmetic Me...,Capital of France is Paris.,"{'Readability and Complexity': {'ARI': 2.7, 'F...",{'Readability and Complexity': {'Arithmetic Me...,Capital of France was Paris.,"{'Readability and Complexity': {'ARI': 3.7, 'F...",{'Readability and Complexity': {'Arithmetic Me...


In [ ]:
result['Model A-Scores'][0]

{'Readability and Complexity': {'ARI': 2.0, 'Flesch-Kincaid Grade Level': 2.1},
 'Language Modeling Performance': {'Perplexity': 15.794241905212402},
 'Text Toxicity': {'Toxicity Level': 0.06541123241186142},
 'Text Similarity and Relevance': {'BLEU': 5.971405938334709e-78,
  'Cosine Similarity': 1.0000000000000002,
  'Semantic Similarity': 0.9698609113693237,
  'Jaccard Similarity': 0.8571428571428571},
 'Information Retrieval': {'Precision': 0.8571428571428571,
  'Recall': 0.8571428571428571,
  'F1-Score': 0.8571428571428571}}

In [ ]:
result['Model A-falcon_Score'][0]

In [ ]:
ModelPerformancePlotter(agg_score_df).get_falcon_performance_quadrant()

In [ ]:
from falcon_evaluate.fevaluate_reliability import Reliability_evaluator

Reliability_eval = Reliability_evaluator()

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/738M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

# Example DataFrame
data = {
    "prompt": ["What is the capital of Portugal?"],
    "reference": ["The capital of Portugal is Lisbon."],
    "Model A": ["Lisbon is the capital of Portugal."],
    "Model B": ["Portugal's capital is Lisbon."],
    "Model C": ["Is Lisbon the main city of Portugal?"]
}
df = pd.DataFrame(data)

In [ ]:
df

,prompt,reference,Model A,Model B,Model C
0,What is the capital of Portugal?,The capital of Portugal is Lisbon.,Lisbon is the capital of Portugal.,Portugal's capital is Lisbon.,Is Lisbon the main city of Portugal?


In [ ]:
results_df = Reliability_eval.predict_hallucination_score(df)

In [ ]:
results_df

,prompt,reference,Model A,Model B,Model C,Model A-reliability-Score,Model B-reliability-Score,Model C-reliability-Score
0,What is the capital of Portugal?,The capital of Portugal is Lisbon.,Lisbon is the capital of Portugal.,Portugal's capital is Lisbon.,Is Lisbon the main city of Portugal?,{'hallucination_score': 1.0},{'hallucination_score': 1.0},{'hallucination_score': 0.22}


- Understanding the Hallucination Scores
- High Score (🟢): Approaches to -> 1 , Indicates strong alignment with the reference, suggesting factual accuracy.
- Low Score (🔴): Approaches to -> 0 , Suggests potential deviations from the factual content of the reference text.


## langfuse

In [ ]:
!pip install langfuse openai -q

In [ ]:
LANGFUSE_SECRET_KEY="sk-lf-992b3a76-ea08-420d-9737-6c7e039fca81"
LANGFUSE_PUBLIC_KEY="pk-lf-625f2f07-dcae-45b7-a85e-e3bb2f01f047"
LANGFUSE_HOST="https://cloud.langfuse.com" # 🇪🇺 EU region

In [ ]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
from langfuse.decorators import observe
from langfuse.openai import openai # OpenAI integration

@observe()
def story():
    return openai.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=100,
        messages=[
          {"role": "system", "content": "You are a great storyteller."},
          {"role": "user", "content": "Once upon a time in a galaxy far, far away..."}
        ],
    ).choices[0].message.content

@observe()
def main():
    return story()

main()

'There existed a mystical planet called Zephyr, known for its tranquil forests and shimmering lakes. The inhabitants of Zephyr were a wise and peaceful species called the Lumarians, who possessed a deep connection with nature and the cosmos.\n\nOne day, a powerful dark force known as the Shadowbringers descended upon Zephyr, casting a dark cloud over the once vibrant planet. The Lumarians, sensing the impending danger, called upon a brave young warrior named Elara to embark on a quest'

In [ ]:
completion = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a very accurate calculator."},
      {"role": "user", "content": "1 + 1 = "}],
)

In [ ]:
completion

ChatCompletion(id='chatcmpl-9OJybP0vODvFNuKU3rAEEXLy71io7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='2', role='assistant', function_call=None, tool_calls=None))], created=1715583689, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=24, total_tokens=25))

In [ ]:
from langfuse import Langfuse

# Create Langfuse client
langfuse = Langfuse(
    secret_key=LANGFUSE_SECRET_KEY,
    public_key=LANGFUSE_PUBLIC_KEY,
    host=LANGFUSE_HOST
)

# Create generation in Langfuse
generation = langfuse.generation(
    name="summary-generation",
    model="gpt-3.5-turbo",
    model_parameters={"maxTokens": "1000", "temperature": "0.9"},
    input=[{"role": "system", "content": "You are a helpful assistant."},
     {"role": "user", "content": "Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals..."}],
    metadata={"interface": "whatsapp"}
)

chat_completion = {
    "completion":"The Q3 OKRs contain goals for multiple teams...",
    "usage":{"input": 50, "output": 49, "unit":"TOKENS"}
}

# update span and sets end_time
generation.end(
    output=chat_completion["completion"],
    usage=chat_completion["usage"],
)

# The SDK executes network requests in the background.
# To ensure that all requests are sent before the process exits, call flush()
# Not necessary in long-running production code
langfuse.flush()

In [ ]:
chat_completion

ChatCompletion(id='chatcmpl-9OK42qeyWFvgmymVBp4QsJE00CK9y', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1715584026, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=9, total_tokens=18))